# python - sentiment score

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import LancasterStemmer, WordNetLemmatizer, PorterStemmer
from wordcloud import WordCloud, STOPWORDS
from textblob import TextBlob

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
cluster1 = pd.read_csv("/content/gdrive/Shared drives/cluster1.csv")
cluster2 = pd.read_csv("/content/gdrive/Shared drives/cluster2.csv")
cluster3 = pd.read_csv("/content/gdrive/Shared drives/cluster3.csv")

cluster1_supp = pd.read_csv("/content/gdrive/Shared drives/cluster1_supp.csv")
cluster2_supp = pd.read_csv("/content/gdrive/Shared drives/cluster2_supp.csv")
cluster3_supp = pd.read_csv("/content/gdrive/Shared drives/cluster3_supp.csv")


In [0]:
## These three columns are manually classified here for demo
cluster1['category'] = cluster1.apply(lambda x: 'self' if x.business_id == 'cYwJA2A6I12KNkm2rtXd5g' else 'competitors',axis = 1)
cluster2['category'] = cluster2.apply(lambda x: 'self' if x.business_id == 'eoHdUeQDNgQ6WYEnP2aiRw' else 'competitors',axis = 1)
cluster3['category'] = cluster3.apply(lambda x: 'self' if x.business_id == 'K7lWdNUhCbcnEvI0NhGewg' else 'competitors',axis = 1)
cluster1['cluster'] = 1
cluster2['cluster'] = 2
cluster3['cluster'] = 3
cluster1['cluster_name'] = 'Gordon Ramsay BurGR'
cluster2['cluster_name'] = 'Serendipity 3'
cluster3['cluster_name'] = 'Wicked Spoon'

In [0]:
cluster = pd.concat([cluster1,cluster2,cluster3])
supp = pd.concat([cluster1_supp,cluster2_supp,cluster3_supp])

In [0]:
cluster = pd.DataFrame(cluster.drop(['Unnamed: 0'],axis=1,inplace=False))
supp = pd.DataFrame(supp.drop(['Unnamed: 0'],axis=1,inplace=False))

In [0]:
all_reviews = pd.merge(cluster, supp, on='business_id')
df = all_reviews[['name', 'business_id', 'date', 'stars', 'text','cluster','cluster_name','category']]

In [0]:
print(all_reviews.shape)
print(all_reviews.columns)
print(df.shape)

(430236, 13)
Index(['business_id', 'date', 'stars', 'text', 'category', 'cluster',
       'cluster_name', 'name', 'categories', 'city', 'postal_code',
       'review_count', 'state'],
      dtype='object')
(430236, 8)


In [0]:
df.head()

,name,business_id,date,stars,text,cluster,cluster_name,category
0,Wolfgang Puck Bar & Grill Las Vegas,N0apJkxIem2E8irTBRKnHw,2015-01-18,2.0,Went here last weekend and was pretty disappoi...,1,Gordon Ramsay BurGR,competitors
1,Wolfgang Puck Bar & Grill Las Vegas,N0apJkxIem2E8irTBRKnHw,2017-01-02,5.0,We ate here for a friend's birthday before see...,1,Gordon Ramsay BurGR,competitors
2,Wolfgang Puck Bar & Grill Las Vegas,N0apJkxIem2E8irTBRKnHw,2016-07-25,5.0,Came here after a concert at MGM...service was...,1,Gordon Ramsay BurGR,competitors
3,Wolfgang Puck Bar & Grill Las Vegas,N0apJkxIem2E8irTBRKnHw,2015-08-19,5.0,Sat at the bar and the bar tender introduced m...,1,Gordon Ramsay BurGR,competitors
4,Wolfgang Puck Bar & Grill Las Vegas,N0apJkxIem2E8irTBRKnHw,2012-10-08,1.0,We visited last week. Be Warned! My husband's...,1,Gordon Ramsay BurGR,competitors


## cleaning

In [0]:
df['text'] = df['text'].astype(str).apply(lambda x: " ".join(x.lower() for x in x.split()))

In [0]:
df['text'] = df['text'].str.replace('[^\w\s]','')

## stop word

In [0]:
stop = stopwords.words('english')
df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

## stemming 

In [0]:
st = PorterStemmer()
df['text'] = df['text'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

## Sentiment Score

In [0]:
def get_polarity(text):
    textblob = TextBlob(text)
    pol = textblob.sentiment.polarity
    return round(pol,3)

#def get_subjectivity(text):
#    textblob = TextBlob(text)
#    subj = textblob.sentiment.subjectivity
#    return round(subj,3)

In [0]:
df['satisfaction'] = df['text'].apply(get_polarity)
#df['subjectivity'] = df['text'].apply(get_subjectivity)

In [0]:
df = pd.DataFrame(df.drop(['text'],axis=1,inplace=False))

In [0]:
df.to_csv('/content/gdrive/Shared drives/SentimentByCluster_plus.csv')